<a href="https://colab.research.google.com/github/Gitjesus22/Rstudio/blob/main/datatable_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table><tr>
<td><img src="https://solariabiodata.com.mx/images/solaria_logo_black.png" alt="Soluciones de siguiente generacion" width=200/>
<td>
<td>
<td><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/1/1b/R_logo.svg/160px-R_logo.svg.png">
</tr></table>


# El paquete `data.table`

Este paquete permite la aceleración cuando se trabaja con archivos muy grandes de datos. De manera general, permite agregación ordenada de datos, modificación de columnas ultrarápida sin la necesidad de generar copias de los archivos.

Debido a la funcionalidad que ofrece para procesar fuentes de datos voluminosas, esta herramienta es de gran utilidad para hacer análisis de datos masivos o _big data_. Es por ello que es uno de los paquetes de `R` más descargados en la actualidad. En el contexto de la bioinformática, resulta crucial tener una herramienta que permita procesar grandes volúmenes de información, que casi siempre resulta ser genómica, por lo cual este paquete es de mucha utilidad.

Tanta es su popularidad que ha inspirado la creación de bibliotecas y módulos para otros lenguajes de programación, `python` es un ejemplo con su paquete `datatable`.


## Instalación y carga

Es un paquete distribuído por el [CRAN](https://cran.r-project.org/) por lo que se puede instalar usando `install.packages()` y cargar con `library()`:

In [ ]:
install.packages("data.table")
library(data.table)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



## Introducción al análisis de datos

La manipulación de datos consiste en operaciones como la selección de conjuntos (_subset_), agrupación de observaciónes (_group_), la actualización de valores (_update_), el agregado de información (_join_), entre otras. Todas esas operaciones están relacionadas entre sí, por lo que si se utilizan todas estas operaciones nos permiten lograr:

 - Una sintáxis concisa y consistente, sin importar qué operaciones se realicen para alcanzar un objetivo
 - Rendimiento fluído en el análisis sin exceso de carga cognitiva por tener que aplicar cada función en un conjunto enorme de datos antes del análisis
 - Optimización automática y efectiva de operaciones internamente, lo cual asegura un uso eficiente de recursos de cómputo.

Este paquete reduce el tiempo de cómputo y de programación de manera considerable.

### Datos

Usaremos un conjunto de datos de los vuelos de la ciudad de Nueva York en el año 2014. Para ello usaremos la función `fread()`:

```R
input<-"https://raw.githubusercontent.com/Rdatatable/data.table/master/vignettes/flights14.csv"
flights <- fread(input)
```




In [ ]:
input<-"https://raw.githubusercontent.com/Rdatatable/data.table/master/vignettes/flights14.csv"
flights <- fread(input)

`fread()` genera una estructura de datos `data.table`, la cual es una extensión de la estructura `data.frame`, la cual es la estructura estándar para almacenamiento de datos en `R`.

Otra forma de generar una estructura `data.table` es mediante la función homónima `data.table`:

```R
DT = data.table(
  ID = c("s","o","l","b","i","o"),
  a = 1:6,
  b = rep(c(TRUE,FALSE),3),
  c = 13:18
)
```

In [ ]:
DT = data.table(
  ID = c("s","o","l","b","i","o"),
  a = 1:6,
  b = rep(c(TRUE,FALSE),3),
  c = 13:18
)
DT

ID,a,b,c
<chr>,<int>,<lgl>,<int>
s,1,TRUE,13
o,2,FALSE,14
l,3,TRUE,15
b,4,FALSE,16
i,5,TRUE,17
o,6,FALSE,18


**Aspectos a notar**

 - A diferencia de un `data.frame`, en `data.table` las columnas con datos de tipo `character` no son convertidos en `factors`
 - `data.table` no utiliza nombres de renglones `row.names`

### Optimización de `data.table`

Este paquete se inspira en operaciones que se pueden realizar en `SQL`, un lenguaje que requiere una estructuración muy fina de los datos.

```
DT[i, j, by]

##   R:                 i                 j        by
## SQL:  where | order by   select | update  group by
```

Esto se interpretaría como

> 1. Toma `DT`
> 2. selecciona/reordena reglones usando `i`
> 3. entonces calcula `j`
> 4. agrupando por `by`.


#### Seleccionando renglones con `i`


```R
ans <- flights[origin == "JFK" & month == 6L]
head(ans)
```

In [ ]:
head(ans)

year,month,day,dep_delay,arr_delay,carrier,origin,dest,air_time,distance,hour
<int>,<int>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<int>,<int>,<int>
2014,6,1,-9,-5,AA,JFK,LAX,324,2475,8
2014,6,1,-10,-13,AA,JFK,LAX,329,2475,12
2014,6,1,18,-1,AA,JFK,LAX,326,2475,7
2014,6,1,-6,-16,AA,JFK,LAX,320,2475,10
2014,6,1,-4,-45,AA,JFK,LAX,326,2475,18
2014,6,1,-6,-23,AA,JFK,LAX,329,2475,14


```R
ans <- flights[1:2]
ans
```

In [ ]:
ans <- flights[1:2]
ans

year,month,day,dep_delay,arr_delay,carrier,origin,dest,air_time,distance,hour
<int>,<int>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<int>,<int>,<int>
2014,1,1,14,13,AA,JFK,LAX,359,2475,9
2014,1,1,-3,13,AA,JFK,LAX,363,2475,11


```R
ans <- flights[order(origin, -dest)]
head(ans)
```

In [ ]:
ans <- flights[order(origin, -dest)]
head(ans)

year,month,day,dep_delay,arr_delay,carrier,origin,dest,air_time,distance,hour
<int>,<int>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<int>,<int>,<int>
2014,1,5,6,49,EV,EWR,XNA,195,1131,8
2014,1,6,7,13,EV,EWR,XNA,190,1131,8
2014,1,7,-6,-13,EV,EWR,XNA,179,1131,8
2014,1,8,-7,-12,EV,EWR,XNA,184,1131,8
2014,1,9,16,7,EV,EWR,XNA,181,1131,8
2014,1,13,66,66,EV,EWR,XNA,188,1131,9


En este caso la función `order()` está optimizada mediante una función interna de `data.table`: `forder()`. 

#### Seleccionando columnas con `j`

Primero obtengamos una columna como un `vector`

```R
ans <- flights[, arr_delay]
head(ans)
```


In [ ]:
ans <- flights[, arr_delay]
class(ans)

[1] "integer"

Para recuperar una columna como un `data.table`

```R
ans <- flights[, list(arr_delay)]
head(ans)
```

In [ ]:
ans <- flights[, list(arr_delay)]
head(ans)

arr_delay
<int>
13
13
9
-26
1
0


la función `list()` asegura que una columna sea regresada como un `data.table`, no obstante es viable utilizar la abreviatura `.()` misma que tiene el mismo efecto.

Pensemos en esta forma como una _expresión j_, mientras en esa expresión se obtengan listas, el resultado será un `data.table`

```R
ans <- flights[, .(arr_delay, dep_delay)]
head(ans)
```

In [ ]:
ans <- flights[, .(arr_delay, dep_delay)]
head(ans)

arr_delay,dep_delay
<int>,<int>
13,14
13,-3
9,2
-26,-8
1,2
0,4


#### Calculando en `j`


Realizar operaciones usando _expresiones j_ es muy sencillo:

```R
ans <- flights[, sum( (arr_delay + dep_delay) < 0 )]
ans
```


In [ ]:
ans <- flights[, sum( (arr_delay + dep_delay) < 0 )]
ans

[1] 141814

Esto se interpreta como calcula cuántos viajes tienen un retraso total menor a 0

#### Seleccionando en `i` y calculando en `j`

Revisemos cuál es el retraso promedio de los arribos y salidas de todos los vuelos del aeropuerto "JFK" en el mes de junio:

```R
ans <- flights[origin == "JFK" & month == 6L,
               .(m_arr = mean(arr_delay), m_dep = mean(dep_delay))]
ans
```

In [ ]:
ans <- flights[origin == "JFK" & month == 6L,
               .(m_arr = mean(arr_delay), m_dep = mean(dep_delay))]
ans

m_arr,m_dep
<dbl>,<dbl>
5.839349,9.807884


Calculemos cuántos viajes se hicieron en 2014 que partieron del aeropuerto "JFK" en junio.

```R
ans <- flights[origin == "JFK" & month == 6L, .N]
ans
```

Nótese que hay un símbolo especial `.N` el cual es una variable especial de `data.table` que contiene el numero de observaciones 

In [ ]:
ans <- flights[origin == "JFK" & month == 6L, .N]
ans

[1] 8422

#### Agrupando usando `by`

Veamos como obtener el conteo de vuelos de diferentes aeropuertos:

```R
ans <- flights[, .(.N), by = .(origin)]
ans
```

In [ ]:
ans <- flights[, .(.N), by = .(origin)]
ans

origin,N
<chr>,<int>
JFK,81483
LGA,84433
EWR,87400


Revisemos el total de vuelos que tiene una aerolínea por ruta:

```R
ans <- flights[carrier == "AA", .N, by = .(origin, dest)]
head(ans)
```

In [ ]:
ans <- flights[carrier == "AA", .N, by = .(origin, dest)]
head(ans)

origin,dest,N
<chr>,<chr>,<int>
JFK,LAX,3387
LGA,PBI,245
EWR,LAX,62
JFK,MIA,1876
JFK,SEA,298
EWR,MIA,848


Ahora calculemos el promedio que tienen los retrasos de una aerolínea en particular y agrupemos por las rutas de vuelo y los meses

```R
ans <- flights[carrier == "AA",
        .(mean(arr_delay), mean(dep_delay)),
        by = .(origin, dest, month)]
ans
```

In [ ]:
ans <- flights[carrier == "AA",
        .(mean(arr_delay), mean(dep_delay)),
        by = .(origin, dest, month)]
ans

origin,dest,month,V1,V2
<chr>,<chr>,<int>,<dbl>,<dbl>
JFK,LAX,1,6.5903614,14.2289157
LGA,PBI,1,-7.7586207,0.3103448
EWR,LAX,1,1.3666667,7.5000000
JFK,MIA,1,15.7206704,18.7430168
JFK,SEA,1,14.3571429,30.7500000
EWR,MIA,1,11.0112360,12.1235955
JFK,SFO,1,19.2522523,28.6396396
JFK,BOS,1,12.9196429,15.2142857
JFK,ORD,1,31.5862069,40.1724138
